# Extracting word surprisal from GPT-2

In [1]:
import math
import csv
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re, os
import bisect
import kenlm
from string import punctuation
import scipy
import os, torch
from transformers import AutoTokenizer, GPT2LMHeadModel
import numpy as np
import pandas as pd

2025-12-15 09:40:42.571546: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 09:40:45.627079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765788046.191070  193758 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765788046.342335  193758 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765788047.941985  193758 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Calculating GPT 2 surprisal and entropy

In [8]:
import pandas as pd

def extract_surprisal(stim):
    #load the sentences (target or distractor)
    sentence_list = pd.read_csv('target_distractor_final_final.csv', sep = ";")
    if stim == "target": 
        sentences = sentence_list['target_sent']
        sentence_id = sentence_list['id']
    else: 
        sentences = sentence_list['distractor_sent']
        sentence_id = sentence_list['id_dis']

        
    
    model = GPT2LMHeadModel.from_pretrained("dbmdz/german-gpt2")
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained("dbmdz/german-gpt2")
    
    softmax = torch.nn.Softmax(dim=-1)
    ctx_size = model.config.max_position_embeddings
    bos_id = model.config.bos_token_id
    sentnum = 0
    
    words_sent = []
    word_suprisal = []
    word_entropy = []
    sentence_ids = []
    word_order = []
    
    #sentences = sentences[40:44]
    for sent in sentences:
        batches = []
        sent = sent.replace('.', '')
        sent = sent.replace(',', '')
        
        words = sent.split(" ")
        id_sentence = sentence_id[sentnum]
    
        tokenizer_output = tokenizer(sent)
        ids = tokenizer_output.input_ids
        attn = tokenizer_output.attention_mask
        start_idx = 0
        word_o = 1
    
        while len(ids) > ctx_size:
            batches.append((torch.tensor([bos_id] + ids[:ctx_size-1]).unsqueeze(0),
                            torch.tensor([1] + attn[:ctx_size-1]).unsqueeze(0), start_idx))
            ids = ids[int(ctx_size/2):]
            attn = attn[int(ctx_size/2):]
            start_idx = int(ctx_size/2) - 1
    
        batches.append((torch.tensor([bos_id] + ids).unsqueeze(0),
                        torch.tensor([1] + attn).unsqueeze(0), start_idx))
    
        curr_word_ix = 0
        curr_word_surp = []
        curr_word_ent = []
        curr_toks = ""
    
        for batch_ids, batch_attn, start_idx in batches:
            
            batch_input = {'input_ids': batch_ids, 'attention_mask': batch_attn}
            with torch.no_grad():
                model_output = model(**batch_input)
    
            logits = model_output.logits.squeeze(0)
            probs = softmax(logits)
            log_probs = torch.log2(probs)
            entropy = -torch.sum(probs * log_probs, dim=-1)  # [seq_len]
            
            target_ids = batch_ids.squeeze(0)[1:]  # shift by one
            toks = tokenizer.convert_ids_to_tokens(batch_ids.squeeze(0))[1:]
            index = torch.arange(len(target_ids))
            surprisal = -torch.log2(softmax(logits)[index, target_ids])
    
            for i in range(start_idx, len(toks)):
                cleaned_tok = toks[i].replace("Ġ", "", 1)
                cleaned_tok = cleaned_tok.replace("Ãľ", "Ü").encode("utf-8").decode("utf-8")
                cleaned_tok = cleaned_tok.replace("Ã¼", "ü").encode("utf-8").decode("utf-8")
                cleaned_tok = cleaned_tok.replace("Ã¶", "ö").encode("utf-8").decode("utf-8")
                cleaned_tok = cleaned_tok.replace("Ã¤", "ä").encode("utf-8").decode("utf-8")
                
                cleaned_tok = cleaned_tok.replace("ÃŁ", "ß").encode("utf-8").decode("utf-8")
                cleaned_tok = cleaned_tok.replace("ÃĦ", "Ä").encode("utf-8").decode("utf-8")
                cleaned_tok = cleaned_tok.replace("Ãĸ", "Ö").encode("utf-8").decode("utf-8")
                cleaned_tok = cleaned_tok.replace("Ã©", "é").encode("utf-8").decode("utf-8")
                
          
    
                curr_word_surp.append(surprisal[i].item())
                curr_word_ent.append(entropy[i].item())
                curr_toks += cleaned_tok
    
                words[curr_word_ix] = words[curr_word_ix].replace(cleaned_tok, "", 1)
                if words[curr_word_ix] == "":
                    words_sent.append(curr_toks)
                    word_suprisal.append(sum(curr_word_surp))
                    word_entropy.append(sum(curr_word_ent))
                    sentence_ids.append(id_sentence)
                    word_order.append(word_o)
    
                    word_o += 1
                    curr_word_surp = []
                    curr_word_ent = []
                    curr_toks = ""
                    curr_word_ix += 1
    
        sentnum += 1


    surprisal_df = pd.DataFrame(zip(words_sent, word_suprisal, word_entropy,sentence_ids, word_order), columns = ['words', 'surprisal','entropy', 'id', 'word_order'])
    surprisal_df['surprisal_new']= np.where(surprisal_df['word_order'] == 1, np.nan, surprisal_df['surprisal'])

    surprisal_df.to_csv("./trf_input/word_surprisal_gpt_{s}.csv".format(s=stim), encoding = "utf-8")

#Word frequency
def extract_word_frequency(stim): 

    subtlex = pd.read_excel("./SUBTLEX-DE cleaned with Google00 frequencies.xlsx")
    target_list = pd.read_csv('./target_distractor_final_final.csv', sep = ";")
    
    subtlex = subtlex.dropna(subset = "Word")
    subtlex = subtlex[['Word', 'lgSUBTLEX']]
    subtlex['Word'] = subtlex['Word'].str.lower()
    
    
   
    
    words_sent = []
    word_freq = []
    sentence_ids = []
    word_order = []
    
    
    
    for i,r in target_list.iterrows():
        
        if stim == "target": 
            sent = r['target_sent']
            ids = r['id']
        else: 
            sent = r['distractor_sent']
            ids = r['id_dis']
            
    

        
        wordlist = sent.split(" ")
        wo = 1
        for w in wordlist:
            w = w.replace(",", "")
            w = w.replace(',', '')
            w = w.replace(':', '')
            w = w.replace('!', '')
            w = w.replace('"', '')
            w = w.replace('.', '')
    
            w = w.lower()
            f_p = subtlex[subtlex['Word'] == w]
    
            if len(f_p) == 0:
                w = w.replace("ß", "ss")
                f_p = subtlex[subtlex['Word'] == w]
    
            if len(f_p) == 1:
                word_freq.append(f_p['lgSUBTLEX'].iloc[0])
            
            elif len(f_p) == 0:
                word_freq.append(np.min(subtlex['lgSUBTLEX']))
            else:
                print(w)
    
            words_sent.append(w)
            sentence_ids.append(ids)
            word_order.append(wo)
            wo = wo + 1
                
    
    word_freq_df = pd.DataFrame(zip(sentence_ids, word_order, words_sent, word_freq), columns = ['id', 'word_order', 'word', 'frequency'])
    word_freq_df.to_csv('./trf_input/word_frequency_{s}_lgsubtlex.csv'.format(s=stim))
        

In [9]:
for stim in ['target', 'dis']:
    print(stim)
    extract_surprisal(stim)
    extract_word_frequency(stim)

target
dis
